<a href="https://colab.research.google.com/github/reiniscimurs/gnn_with_pytorch/blob/main/section_3/02_simple_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of a Simple Graph Neural Network (GNN)
Implement a simple Graph Neural Network (GNN) using PyTorch Geometric. Since we'll be using GPU for training, go to "Edit" -> "Notebook Settings" and select "GPU" under "Hardware Accelerator."

## Installation of PyTorch Geometric
Install the library "PyTorch Geometric" for Graph Neural Networks, along with its related libraries.

In [ ]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

## Loading the Dataset
We will load the dataset "Cora." The Cora dataset consists of a citation network of 2,708 scientific papers. Represented as a graph, each node corresponds to a paper, and edges represent citation relationships. Each paper is classified into one of seven classes. Each paper in the dataset has a word vector as a feature, indicating the presence (1) or absence (0) of words. The number of unique words is 1433.

The following code loads the Cora dataset:

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="/tmp/Cora", name="Cora")
data = dataset[0]

Define a function to display information about the graph.

In [ ]:
def graph_info(data):

    print("Number of nodes:", data.num_nodes)
    print("Number of edges:", data.num_edges)
    print("Number of features:", data.num_node_features)
    print("Is the graph undirected?", data.is_undirected())
    print("Are there isolated nodes?", data.has_isolated_nodes())
    print("Are there self-loops?", data.has_self_loops())

    print()

    print("Keys: ", data.keys)
    print("Node features for each node")
    print(data["x"])
    print("Labels for each node")
    print(data["y"])
    print("Edges")
    print(data["edge_index"])

Use the function to display information about the dataset.

In [ ]:
graph_info(data)

Visualize the graph using NetworkX and Matplotlib.

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

data_nx = to_networkx(data)  # Convert to a NetworkX graph

plt.figure(figsize=(12, 10))
nx.draw(data_nx,
        node_color=data.y,
        node_size=10)
plt.show()

## Model Construction
We will build a model for a simple Graph Neural Network (GNN). This time, we'll implement a "Graph Convolutional Networks" with two layers. For the layer implementation, we'll use `GCNConv()`.
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 32)
        self.relu = nn.ReLU()  # ReLU
        self.conv2 = GCNConv(32, dataset.num_classes)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index

        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)

        return x

net = GCN()
net.cuda()  # Enable GPU

## Train
Train the model using the training data.

In [ ]:
from torch import optim

data = data.cuda()  # GPU support

# Cross-entropy loss function
loss_fnc = nn.CrossEntropyLoss()

# Optimization algorithm
optimizer = optim.Adam(net.parameters())

net.train()  # Training mode
for epoch in range(200):
    optimizer.zero_grad()  # ① Initialize gradients
    out = net(data)  # ② Obtain predictions through forward propagation
    loss = loss_fnc(out[data.train_mask], data.y[data.train_mask])  # ③ Calculate loss from predictions and correct labels

    loss.backward()  # ④ Compute gradients through backpropagation from the loss
    optimizer.step()  # ⑤ Update parameters through the optimization algorithm

## Model Evaluation
Evaluate the model by measuring accuracy using test nodes.

In [ ]:
net.eval()  # Evaluation mode
pred = net(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
accuracy = int(correct) / int(data.test_mask.sum())
print("Accuracy:", str(accuracy * 100) + "%")